In [1]:
import os
import json
import numpy as np
from sklearn.model_selection import cross_validate
import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_validate
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw/sample_meta.csv')

In [3]:
df.shape

(1890, 27)

In [4]:
test_set = np.array(df.index[df['fold'] == 'test'])

In [5]:
# function to return the cv splits by index from sample_meta.csv

def fetch_cv_indicies(df):
    unique_folds = list(df.fold.unique())
    unique_folds.remove('test')
    # print(unique_folds)
    
    cv_splits = []
    
    for fold in unique_folds:
        
        left = np.array(df.index[df['fold'] == fold])
        
        right = np.array(df.index[(df['fold'] != fold) & (df['fold'] != 'test')])
        
        #print(f'Fold: {fold}\nLeft: {len(left)}\nRight: {len(right)}\nTotal: {(len(left)+len(right))}\n\n')
        
        split = (left, right)
        
        cv_splits.append(split)
        
    return cv_splits



In [6]:
cv_splits = fetch_cv_indicies(df)

In [7]:
mfcc_path = '../data/processed/13_mfcc_5.json'

In [8]:
def load_data(data_path, target_labels):
    
    with open(data_path, "r") as fp:
        data = json.load(fp)
        
    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data[target_labels])

    print("Data succesfully loaded!")

    return  X, y

In [9]:
# load data
X, y = load_data(data_path=mfcc_path, target_labels='class_label')

Data succesfully loaded!


In [10]:
def build_model():
    model = keras.Sequential([

        # input layer
        keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

        # 1st dense layer
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.2),

        # 2nd dense layer
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.2),

        # 3rd dense layer
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.2),

        # output layer
        keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='BinaryCrossentropy',
                  metrics=['binary_accuracy'])
    
    return model

In [11]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.FATAL)


In [12]:
# evaluate model with cv splits
build = build_model()
model = KerasClassifier(model=build, batch_size=32, epochs=50, verbose=1)

results = cross_validate(model, X, y, cv=cv_splits)
print("Baseline: %.2f%% (%.2f%%)" % (results['test_score'].mean()*100, results['test_score'].std()*100))



Epoch 1/50
6/6 [==============================] - 1s 22ms/step - loss: 40.6002 - binary_accuracy: 0.7529
Epoch 2/50
6/6 [==============================] - 0s 23ms/step - loss: 1.2820 - binary_accuracy: 1.0000
Epoch 3/50
6/6 [==============================] - 0s 23ms/step - loss: 1.2620 - binary_accuracy: 1.0000
Epoch 4/50
6/6 [==============================] - 0s 24ms/step - loss: 1.2425 - binary_accuracy: 1.0000
Epoch 5/50
6/6 [==============================] - 0s 25ms/step - loss: 1.2235 - binary_accuracy: 1.0000
Epoch 6/50
6/6 [==============================] - 0s 25ms/step - loss: 1.2051 - binary_accuracy: 1.0000
Epoch 7/50
6/6 [==============================] - 0s 25ms/step - loss: 1.1873 - binary_accuracy: 1.0000
Epoch 8/50
6/6 [==============================] - 0s 25ms/step - loss: 1.1701 - binary_accuracy: 1.0000
Epoch 9/50
6/6 [==============================] - 0s 25ms/step - loss: 1.1534 - binary_accuracy: 1.0000
Epoch 10/50
6/6 [==============================] - 0s 29ms/step

Epoch 1/50
6/6 [==============================] - 1s 26ms/step - loss: 54.5118 - binary_accuracy: 0.6667
Epoch 2/50
6/6 [==============================] - 0s 23ms/step - loss: 27.1330 - binary_accuracy: 0.7840
Epoch 3/50
6/6 [==============================] - 0s 24ms/step - loss: 29.4282 - binary_accuracy: 0.6914
Epoch 4/50
6/6 [==============================] - 0s 24ms/step - loss: 15.8921 - binary_accuracy: 0.8333
Epoch 5/50
6/6 [==============================] - 0s 23ms/step - loss: 14.5582 - binary_accuracy: 0.8333
Epoch 6/50
6/6 [==============================] - 0s 24ms/step - loss: 9.1305 - binary_accuracy: 0.8642
Epoch 7/50
6/6 [==============================] - 0s 31ms/step - loss: 9.1578 - binary_accuracy: 0.8519
Epoch 8/50
6/6 [==============================] - 0s 25ms/step - loss: 8.0746 - binary_accuracy: 0.9012
Epoch 9/50
6/6 [==============================] - 0s 26ms/step - loss: 7.7568 - binary_accuracy: 0.8951
Epoch 10/50
6/6 [==============================] - 0s 26ms/

Epoch 1/50
7/7 [==============================] - 1s 23ms/step - loss: 66.7402 - binary_accuracy: 0.6904
Epoch 2/50
7/7 [==============================] - 0s 23ms/step - loss: 25.9351 - binary_accuracy: 0.8173
Epoch 3/50
7/7 [==============================] - 0s 24ms/step - loss: 20.0971 - binary_accuracy: 0.7665
Epoch 4/50
7/7 [==============================] - 0s 23ms/step - loss: 8.9462 - binary_accuracy: 0.8883
Epoch 5/50
7/7 [==============================] - 0s 23ms/step - loss: 10.7562 - binary_accuracy: 0.8376
Epoch 6/50
7/7 [==============================] - 0s 26ms/step - loss: 7.3455 - binary_accuracy: 0.9036
Epoch 7/50
7/7 [==============================] - 0s 27ms/step - loss: 6.5221 - binary_accuracy: 0.9086
Epoch 8/50
7/7 [==============================] - 0s 30ms/step - loss: 8.0623 - binary_accuracy: 0.8629
Epoch 9/50
7/7 [==============================] - 0s 27ms/step - loss: 6.3194 - binary_accuracy: 0.8985
Epoch 10/50
7/7 [==============================] - 0s 26ms/s

Epoch 1/50
6/6 [==============================] - 2s 31ms/step - loss: 59.6263 - binary_accuracy: 0.6545
Epoch 2/50
6/6 [==============================] - 0s 31ms/step - loss: 31.8086 - binary_accuracy: 0.9333
Epoch 3/50
6/6 [==============================] - 0s 28ms/step - loss: 28.1465 - binary_accuracy: 0.9333
Epoch 4/50
6/6 [==============================] - 0s 25ms/step - loss: 17.0506 - binary_accuracy: 0.9333
Epoch 5/50
6/6 [==============================] - 0s 25ms/step - loss: 10.3914 - binary_accuracy: 0.9212
Epoch 6/50
6/6 [==============================] - 0s 28ms/step - loss: 13.5945 - binary_accuracy: 0.8424
Epoch 7/50
6/6 [==============================] - 0s 25ms/step - loss: 12.0725 - binary_accuracy: 0.8667
Epoch 8/50
6/6 [==============================] - 0s 25ms/step - loss: 9.4123 - binary_accuracy: 0.9273
Epoch 9/50
6/6 [==============================] - 0s 28ms/step - loss: 8.0198 - binary_accuracy: 0.9091
Epoch 10/50
6/6 [==============================] - 0s 25m

Epoch 1/50
6/6 [==============================] - 1s 21ms/step - loss: 48.0575 - binary_accuracy: 0.5424
Epoch 2/50
6/6 [==============================] - 0s 23ms/step - loss: 29.0193 - binary_accuracy: 0.6497
Epoch 3/50
6/6 [==============================] - 0s 21ms/step - loss: 17.5965 - binary_accuracy: 0.7910
Epoch 4/50
6/6 [==============================] - 0s 22ms/step - loss: 14.8438 - binary_accuracy: 0.8418
Epoch 5/50
6/6 [==============================] - 0s 22ms/step - loss: 10.0327 - binary_accuracy: 0.8079
Epoch 6/50
6/6 [==============================] - 0s 23ms/step - loss: 7.3035 - binary_accuracy: 0.8305
Epoch 7/50
6/6 [==============================] - 0s 23ms/step - loss: 6.3452 - binary_accuracy: 0.8644
Epoch 8/50
6/6 [==============================] - 0s 23ms/step - loss: 5.0690 - binary_accuracy: 0.9040
Epoch 9/50
6/6 [==============================] - 0s 27ms/step - loss: 8.0014 - binary_accuracy: 0.8870
Epoch 10/50
6/6 [==============================] - 0s 24ms/

Epoch 1/50
5/5 [==============================] - 1s 26ms/step - loss: 48.6274 - binary_accuracy: 0.6646
Epoch 2/50
5/5 [==============================] - 0s 25ms/step - loss: 25.3064 - binary_accuracy: 0.7975
Epoch 3/50
5/5 [==============================] - 0s 27ms/step - loss: 14.2330 - binary_accuracy: 0.8101
Epoch 4/50
5/5 [==============================] - 0s 25ms/step - loss: 7.0749 - binary_accuracy: 0.8924
Epoch 5/50
5/5 [==============================] - 0s 26ms/step - loss: 6.1100 - binary_accuracy: 0.9241
Epoch 6/50
5/5 [==============================] - 0s 22ms/step - loss: 2.0295 - binary_accuracy: 0.9810
Epoch 7/50
5/5 [==============================] - 0s 26ms/step - loss: 3.3880 - binary_accuracy: 0.9557
Epoch 8/50
5/5 [==============================] - 0s 26ms/step - loss: 3.2739 - binary_accuracy: 0.9747
Epoch 9/50
5/5 [==============================] - 0s 26ms/step - loss: 1.6232 - binary_accuracy: 0.9810
Epoch 10/50
5/5 [==============================] - 0s 27ms/st

Epoch 1/50
6/6 [==============================] - 1s 21ms/step - loss: 11.6091 - binary_accuracy: 0.9149
Epoch 2/50
6/6 [==============================] - 0s 21ms/step - loss: 12.3080 - binary_accuracy: 0.9574
Epoch 3/50
6/6 [==============================] - 0s 20ms/step - loss: 5.5218 - binary_accuracy: 0.9628
Epoch 4/50
6/6 [==============================] - 0s 20ms/step - loss: 3.6505 - binary_accuracy: 0.9521
Epoch 5/50
6/6 [==============================] - 0s 22ms/step - loss: 3.3443 - binary_accuracy: 0.9362
Epoch 6/50
6/6 [==============================] - 0s 25ms/step - loss: 2.4457 - binary_accuracy: 0.9734
Epoch 7/50
6/6 [==============================] - 0s 23ms/step - loss: 2.8803 - binary_accuracy: 0.9468
Epoch 8/50
6/6 [==============================] - 0s 23ms/step - loss: 1.8020 - binary_accuracy: 0.9787
Epoch 9/50
6/6 [==============================] - 0s 23ms/step - loss: 1.4851 - binary_accuracy: 0.9787
Epoch 10/50
6/6 [==============================] - 0s 25ms/ste

Epoch 1/50
6/6 [==============================] - 1s 23ms/step - loss: 38.6807 - binary_accuracy: 0.5805
Epoch 2/50
6/6 [==============================] - 0s 26ms/step - loss: 7.7499 - binary_accuracy: 0.8966
Epoch 3/50
6/6 [==============================] - 0s 25ms/step - loss: 3.7280 - binary_accuracy: 0.9425
Epoch 4/50
6/6 [==============================] - 0s 25ms/step - loss: 2.2788 - binary_accuracy: 0.9655
Epoch 5/50
6/6 [==============================] - 0s 26ms/step - loss: 1.6240 - binary_accuracy: 0.9828
Epoch 6/50
6/6 [==============================] - 0s 26ms/step - loss: 1.3781 - binary_accuracy: 0.9828
Epoch 7/50
6/6 [==============================] - 0s 27ms/step - loss: 3.0346 - binary_accuracy: 0.9770
Epoch 8/50
6/6 [==============================] - 0s 25ms/step - loss: 1.6668 - binary_accuracy: 0.9885
Epoch 9/50
6/6 [==============================] - 0s 28ms/step - loss: 1.1849 - binary_accuracy: 1.0000
Epoch 10/50
6/6 [==============================] - 0s 30ms/step

Epoch 1/50
6/6 [==============================] - 2s 27ms/step - loss: 15.7485 - binary_accuracy: 0.7977
Epoch 2/50
6/6 [==============================] - 0s 27ms/step - loss: 1.2846 - binary_accuracy: 1.0000
Epoch 3/50
6/6 [==============================] - 0s 27ms/step - loss: 1.2664 - binary_accuracy: 1.0000
Epoch 4/50
6/6 [==============================] - 0s 26ms/step - loss: 1.2486 - binary_accuracy: 1.0000
Epoch 5/50
6/6 [==============================] - 0s 29ms/step - loss: 1.2313 - binary_accuracy: 1.0000
Epoch 6/50
6/6 [==============================] - 0s 30ms/step - loss: 1.2145 - binary_accuracy: 1.0000
Epoch 7/50
6/6 [==============================] - 0s 30ms/step - loss: 1.1982 - binary_accuracy: 1.0000
Epoch 8/50
6/6 [==============================] - 0s 30ms/step - loss: 1.1825 - binary_accuracy: 1.0000
Epoch 9/50
6/6 [==============================] - 0s 30ms/step - loss: 1.1672 - binary_accuracy: 1.0000
Epoch 10/50
6/6 [==============================] - 0s 32ms/step

Epoch 1/50
5/5 [==============================] - 1s 28ms/step - loss: 40.6162 - binary_accuracy: 0.5103
Epoch 2/50
5/5 [==============================] - 0s 25ms/step - loss: 21.1107 - binary_accuracy: 0.6483
Epoch 3/50
5/5 [==============================] - 0s 27ms/step - loss: 10.6601 - binary_accuracy: 0.7517
Epoch 4/50
5/5 [==============================] - 0s 27ms/step - loss: 7.6316 - binary_accuracy: 0.8414
Epoch 5/50
5/5 [==============================] - 0s 31ms/step - loss: 6.8831 - binary_accuracy: 0.8138
Epoch 6/50
5/5 [==============================] - 0s 31ms/step - loss: 5.8727 - binary_accuracy: 0.8345
Epoch 7/50
5/5 [==============================] - 0s 32ms/step - loss: 6.5728 - binary_accuracy: 0.8552
Epoch 8/50
5/5 [==============================] - 0s 33ms/step - loss: 5.5566 - binary_accuracy: 0.8690
Epoch 9/50
5/5 [==============================] - 0s 30ms/step - loss: 5.0715 - binary_accuracy: 0.8621
Epoch 10/50
5/5 [==============================] - 0s 32ms/st